In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Motif Extraction

Define the function

In [ ]:
#%%
import pandas as pd
from google import genai
from PIL import Image
import io
import numpy as np
import time
import re
import matplotlib.pyplot as plt



def load_image(image_path):
    with open(image_path, "rb") as img_file:
        image = Image.open(io.BytesIO(img_file.read()))
        if image_path.endswith(".tif"):
            # change to numpy array
            img_array = np.array(image)

            # normalization 0~255
            img_8bit = (255 * (img_array / img_array.max())).astype(np.uint8)


            image = Image.fromarray(img_8bit)
        return image



In [ ]:
import os
import shutil

def clear_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.remove(file_path)  # Clean the folder
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Delete folder
        except Exception as e:
            print(f'删除失败: {file_path}. 原因: {e}')


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.patches as patches
import os

def identify_motif(image_path, boxes, label, ID, dynasty, save_dir="motif_crops"):
    # Load Image
    image = Image.open(image_path)
    width, height = image.size
    scale_x = width / 1024
    scale_y = height / 1024

    # Create folder
    os.makedirs(save_dir, exist_ok=True)






    for i, det in enumerate(boxes):

        if 'flower' in label[i] or 'floral' in label[i]:
          box = det["box_2d"]
          if len(box)==4:
        # obtain the coordinates
            y1, x1, y2, x2 = box
            y1 = int(y1 * scale_y)
            x1 = int(x1 * scale_x)
            y2 = int(y2 * scale_y)
            x2 = int(x2 * scale_x)

            # Clip and save
            crop = image.crop((x1, y1, x2, y2))
            crop_filename = os.path.join(save_dir, f"{dynasty}_{ID}_{i}.png")
            crop.save(crop_filename)
            break



In [ ]:
def translate_label(label,translation_map):
    label_str = str(label).strip()


    if label_str in translation_map:
        return translation_map[label_str]


    for cn, en in sorted(translation_map.items(), key=lambda x: len(x[0]), reverse=True):
        if cn in label_str:
            return en


    return label_str

In [ ]:
import ast
def safe_literal_eval(val):
    if pd.isna(val):  # 或者: if val != val
        return None  #
    try:
        return ast.literal_eval(val)
    except Exception as e:
        print(f"解析失败: {val}, 错误: {e}")
        return None

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/History/Gemini_Resulits/book_qing.csv')
folder_name='/content/drive/MyDrive/History/Merged_Dataset/liaojin'
imagefile=os.path.join(folder_name,df['ID'].values[0]+'.jpg')
df['boxes']=df['boxes'].apply(safe_literal_eval)

In [ ]:
# Translate Map
translation_map = {
    '云': 'cloud',
    '花': 'flower',
    '牡丹': 'peony',
    '蝴蝶': 'butterfly',
    '鹤': 'crane',
    '凤凰': 'phoenix',
    '龙': 'dragon',
    '凤': 'phoenix',
    '山': 'mountain',
    '水波': 'wave',
    '人物': 'figure',
    '鸟': 'bird',
    '梅': 'plum blossom',
    '竹': 'bamboo',
    '鱼': 'fish',
    '鹿': 'beast',
    '虎': 'tiger',
    '狮': 'lion',
    '莲': 'lotus flower',
    '莲花': 'lotus flower',
    '菊花':'chrysanthemum flower',
    '瓣窠': 'petal cluster',
    '瓣窠纹': 'petal cluster',
    '瓣窠式': 'petal cluster',
    '芙蓉': 'hibiscus',
    '团纹': 'medallion',
    '博古': 'antiquities',
    # '博古纹': 'antiquities',
    '赤兔': 'red hare',
    '福': 'fu',
    '直线': 'straight line',
    '葡萄': 'grape',
    '锯齿球形': 'serrated sphere',
    '凤尾': 'phoenix tail',
    '蟠螭纹': 'coiled dragon',
    '方胜': 'square victory',
    '飞燕': 'flying swallow',
    '对马纹': 'paired horse',
    '仕女': 'court lady',
    '佛手': "Buddha's hand",
    '萱草纹': 'daylily',
    '童子骑羊': 'child riding goat',
    '锦鸡': 'golden pheasant',
    '金锭': 'gold ingot',
    '童子': 'child motif',
    '牛形动物纹样': 'ox-shaped animal',
    '夹缬': 'resist-dyed',
    '婴戏图': 'baby play scene',
    '如意吉祥纹': 'ruyi auspicious',
    '缠枝叶': 'intertwined branch and leaf',
    '斗牛纹': 'bullfighting',
    '柿蒂窠': 'persimmon calyx',
    '婴戏': 'baby play',
    '万字': 'swastika',
    '石榴': 'pomegranate',
    '对桃': 'paired peach',
    '奔兽': 'beast',
    '凤纹': 'phoenix',
    '海棠': 'begonia flower',
    '蝙蝠': 'bat',
    '琐纹': 'small linked',
    '盘长': 'endless knot',
    '盘凤': 'coiled phoenix',
    '海水纹样': 'sea wave',
    '球路流苏纹': 'ball-and-tassel',
    '翟纹': 'pheasant',
    '毬路': 'ball',
    '灵芝': 'lingzhi',
    '龟背纹': 'tortoise shell',
    '波斯文': 'Persian script',
    '缠枝': 'intertwined branches',
    '五湖四海纹': 'five lakes and four seas',
    '王': 'king symbol',
    '叶子': 'leaf',
    '菱格': 'rhombus',
    '葫芦': 'gourd',
    '蟒': 'python',
    '雁': 'wild goose',
    '龟甲纹': 'turtle shell',
    '卍': 'swastika',
    # '卍字': 'swastika',
    # '卍 字': 'swastika',
    # '卍字纹': 'swastika',
    '团窠': 'clustered medallion',
    '流水': 'flowing water',
    '灯笼': 'lantern',
    '杂宝': 'assorted treasures',
    '枝干': 'branches',
    # '寿字': 'longevity symbol',
    '寿': 'longevity symbol',
    '蝶': 'butterfly',
    '松枝': 'pine branch',
    '四合如意纹': 'four-in-one ruyi',
    # '力芬': 'lifen', 感觉是错误的可以delete
    '盘球纹': 'coiled ball',
    '联珠团窠': 'beaded clustered medallion',
    '柿蒂': 'persimmon calyx',
    '钱纹': 'coin',
    '八吉祥': 'eight auspicious symbols',
    '拱璧纹': 'arched jade disc',
    '卷草': 'scrolling grass',
    '卷草如意纹': 'scrolling grass',
    '梅竹纹样': 'plum and bamboo',
    '小窠宝花纹锦': 'small-cluster precious flower brocade',
    '宝花':'precious flower',
    '梅花': 'plum blossom flower',
    '盘长': 'endless knot',
    '折枝花卉纹样': 'broken branch floral',
    '赤兔纹': 'red hare',
    '对狮': 'paired lion',
    '灵芝云': 'lingzhi cloud',
    '喜相逢团纹': 'happy meeting medallion',
    '芙蓉花': 'hibiscus flower',
    '牡丹花': 'peony flower',
    '落花': 'falling flower',
    '宝相花': 'precious lotus flower',
    '圆芯朵梅纹': 'round-core plum blossom',
    '缠枝牡丹纹': 'intertwined peony flower',
    '串枝莲花纹': 'clustered lotus flower',
    '翟鸟': 'pheasant bird',
    '葫芦花': 'gourd flower',
    '团凤': 'paired phoenix',
    '波浪花纹': 'wave pattern',
    '波纹': 'wave pattern',
    '锁子纹': 'chain pattern',
    '兽': 'beast',

    '罐': 'pot',
    '鸂鶒': 'mandarin duck',
    '黑缎镶边': 'black satin edging',
    '土': 'earth / soil',
    '扇子': 'fan',
    '勾连纹': 'interlinked pattern',
    '盖': 'lid / cover',
    '阴阳叉': 'yin-yang fork',
    '日': 'sun',
    '胡旋舞人': 'Sogdian dancer',
    '江涯海水与桃树': 'seashore, water, and peach tree',
    '练雀': 'sparrow',
    '圆补': 'round patch',
    '‘富’‘贵’‘万岁’等文字纹样': 'character motifs',
    '飞鹰': 'flying eagle',
    '海水江崖': 'sea and cliff',
    '栏杆裙': 'railing-skirt',
    '宝瓶': 'precious vase',
    '圆盒': 'round box',
    '书籍': 'book / books',
    '如意': 'ruyi',
    '拱桥': 'arched bridge',
    '建筑': ' building',
    '波光粼粼的水面': 'shimmering water surface',
    '楼阁': 'pavilion',
    '撑船': 'boating',
    '孔雀': 'peacock',
    '孔 雀': 'peacock',
    '盆景': 'bonsai',
    '摆件': 'ornament',
    '谷穗': 'ear of grain',
    '水浪': 'water wave',
    '仙人': 'immortal / celestial being',
    # '道家帽子': 'Taoist hat',
    # '汉式长袍': 'Han-style robe', 同理，感觉也不是纹样
    '羽扇': 'feather fan',
    '拱 璧 纹': 'arched jade disc motif',
    '双矩': 'double square / double ruler',
    '鹭鸶': 'egret',
    '菱纹格': 'rhombus grid',
    '麒麟纹': 'qilin motif',
    '方搭纹': 'square lattice pattern',
    '月亮': 'moon',
    '日纹': 'sun',
    '桂树': 'cassia tree',
    '双胜纹': 'double-victory motif', # 不确定的翻译
    '窠形': 'cluster shape',
    '一年景': 'four seasons scene',
    '独 角 麒 麟': 'one-horned qilin',
    '海东青': 'gyrfalcon',
    '奔兔': 'running rabbit',
    '孔雀纹': 'peacock motif',
    # '纹': 'motif / pattern', # 无意义，可以删除
    '猪头': 'pig head',
    # '絹裤残片': 'silk trouser fragment', #不是纹样
    '珊瑚': 'coral',
    '书卷': 'book roll',
    '犀角': 'rhinoceros horn',
    '绿叶': 'green leaf',
    '海 螺': 'conch',
    '联珠纹': 'beaded motif',
    '回纹装饰': 'meander decoration',
    '叶': 'leaf',
    '联珠圈': 'beaded circle',
    '玉树': 'jade tree',
    # '圆领': 'round collar', 不是纹样
    '阿拉伯文': 'Arabic script',
    # '高筒绢袜': 'tall silk socks', 不是纹样
    # '罗鞋': 'silk shoes', # 不是纹样
    '秋': 'character motif',
    '岁': 'character motif',
    '千': 'character motif',
    '万': 'character motif',
    '球路': 'ball-shaped motif',
    '四 合 如 意 纹': 'four-in-one ruyi motif',
    '联珠': 'beaded motif',
    '异兽': 'exotic beast',
    '鹊': 'magpie',
    '带式联珠纹': 'belt-like beaded motif',
    '联珠纹样': 'beaded motif',
    '摩羯': 'makara (mythical creature)',
    '海水': 'sea wave',
    '玉兔': 'jade rabbit',

    '团花': 'clustered flower',
    '十字宾花': 'cross guest flower',
    '菱形方格纹': 'rhombus lattice pattern',
    '描金折枝花纹': 'gold-painted broken branch floral pattern',
    '蓮花': 'lotus flower',
    '桃形纹': 'peach-shaped pattern',
    '花蕾': 'flower bud',
    '海螺': 'conch',
    '金刚杵': 'vajra',
    '菱形': 'rhombus',
    '独角麒麟': 'one-horned qilin',
    '四季花': 'four seasons flowers',
    '小花纹': 'small flower pattern',
    '十字花纹': 'cross pattern',
    '拱璧纹': 'arched jade disc motif',
    '大洋花': 'ocean flower',
    '团荷花双喜字纹': 'clustered lotus with double happiness character pattern',
    '孔雀': 'peacock',
    '皮球花': 'ball flower',
    '花蓝': 'flower basket',
    '花草': 'flowers and grasses',
    'floral pattern':'floral',
    'floral motif':'floral',
    'clouds':'cloud'

}




In [ ]:
folder='/content/drive/MyDrive/History/Merged_Dataset/qing'
for filename in os.listdir(folder):
    if filename.endswith(".jpg"):
        name, ext = os.path.splitext(filename)
        # Clean the name
        new_name = name.rstrip() + ext
        if new_name != filename:
            old_path = os.path.join(folder, filename)
            new_path = os.path.join(folder, new_name)
            print(f"Renaming: '{filename}' → '{new_name}'")
            os.rename(old_path, new_path)









Extract Motif

In [ ]:
motif_folder='/content/drive/MyDrive/History/Motif_Crops/Cloud'
clear_folder(motif_folder)

In [ ]:
gemini_folder='/content/drive/MyDrive/History/Gemini_Resulits/'
book_folder='/content/drive/MyDrive/History/Book_Dataset/'
museum_folder='/content/drive/MyDrive/History/Merged_Dataset/'
motif_folder='/content/drive/MyDrive/History/Motif_Crops/Flower'

for file in os.listdir(gemini_folder):
    # Read Data
    if file.endswith('.csv'):

        df=pd.read_csv(os.path.join(gemini_folder,file))
        df['boxes']=df['boxes'].apply(safe_literal_eval)

        # Translation & Cleaning
        dataset_name = os.path.splitext(file)[0].split('_')[0]
        dynasty=os.path.splitext(file)[0].split('_')[1]

        if dataset_name=='book':
            folder_name=os.path.join(book_folder,dynasty)
        else:
            folder_name=os.path.join(museum_folder,dynasty)
            #filelist=os.listdir(folder_name)



        for index,row in df.iterrows():

          if row['boxes']:
            label_list = [box['label'] for box in row['boxes']]
            box_labels_df = pd.DataFrame({'original_label': label_list})
            box_labels_df['normalized_label'] = box_labels_df['original_label'].str.lower()
            box_labels_df['translated_label'] = box_labels_df['normalized_label'].apply(lambda x: translate_label(x, translation_map))
            image_path=os.path.join(folder_name,row['ID'].strip()+'.jpg')
            #image_path=find_image_path(folder_name,row['ID'])
            ID=row['ID']
            boxes=row['boxes']
            label=box_labels_df['translated_label'].values
            identify_motif(image_path, boxes, label, row['ID'],dynasty, motif_folder)







Clusteting

In [ ]:
motif_folder='/content/drive/MyDrive/History/Motif_Crops/Flower'

In [ ]:
import os
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
import umap
from transformers import AutoImageProcessor, AutoModel
from torchvision import transforms




# Load the model
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = AutoModel.from_pretrained("facebook/dinov2-base")
model.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Dinov2Model(
  (embeddings): Dinov2Embeddings(
    (patch_embeddings): Dinov2PatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): Dinov2Encoder(
    (layer): ModuleList(
      (0-11): 12 x Dinov2Layer(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attention): Dinov2Attention(
          (attention): Dinov2SelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
          )
          (output): Dinov2SelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (layer_scale1): Dinov2LayerScale()
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06,

In [ ]:
!pip install hdbscan

In [ ]:
def load_and_prepare_image(img_path):
    img = Image.open(img_path).convert("RGB")
    #
    return processor(images=img, return_tensors="pt")


In [ ]:
def extract_features(image_paths):
    features = []
    for path in image_paths:
        inputs = load_and_prepare_image(path)
        with torch.no_grad():
            output = model(**inputs).last_hidden_state[:, 0, :]  # CLS token
        features.append(output.squeeze(0).numpy())
    return np.array(features)

In [ ]:
def visualize_clusters(embedding, image_paths, labels):
    plt.figure(figsize=(20, 20))
    for i, (x, y) in enumerate(embedding):
        img = Image.open(image_paths[i]).resize((32, 32))
        plt.imshow(img, extent=(x-0.5, x+0.5, y-0.5, y+0.5), zorder=2)
    plt.scatter(embedding[:, 0], embedding[:, 1], c=labels, alpha=0.5, s=10, zorder=1)
    plt.title("DINOv2 图案特征聚类（UMAP 降维）")
    plt.axis("off")
    plt.show()

In [ ]:
import hdbscan
import umap

def cluster_with_hdbscan(features):
    # Clustring
    reducer = umap.UMAP()
    embedding = reducer.fit_transform(features)

    clusterer = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=3)
    labels = clusterer.fit_predict(embedding)

    return labels, embedding

In [ ]:

def run_dinov2_clustering(image_folder, n_clusters=5):
    image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder)
                   if f.lower().endswith(('.jpg', '.png'))]

    print(f"加载图像数量：{len(image_paths)}")
    features = extract_features(image_paths)

    print("开始聚类...")
    kmeans = KMeans(n_clusters=n_clusters, random_state=20)
    labels = kmeans.fit_predict(features)

    cluster_dict = {}
    for path, label in zip(image_paths, labels):
        cluster_dict.setdefault(f"cluster_{label}", []).append(path)

    # Print the clustering list
    for cluster_name, paths in cluster_dict.items():
        print(f"\n🔹 {cluster_name} 包含 {len(paths)} 张图像：")
        for p in paths:
            print("  -", os.path.basename(p))

    return image_paths,features, cluster_dict

In [ ]:
import math
def show_images_by_cluster(cluster_dict, images_per_row=5):
    for cluster_name, paths in cluster_dict.items():
        print(f"\n🔹 {cluster_name}（共 {len(paths)} 张图像）")

        n_images = len(paths)
        n_rows = math.ceil(n_images / images_per_row)
        plt.figure(figsize=(images_per_row * 2, n_rows * 2))

        for i, path in enumerate(paths):
            img = Image.open(path).convert("RGB")
            plt.subplot(n_rows, images_per_row, i + 1)
            plt.imshow(img)
            plt.axis("off")
            plt.title(f"{os.path.basename(path)}", fontsize=8)

        plt.suptitle(f"{cluster_name}", fontsize=16)
        plt.tight_layout()
        plt.show()

In [ ]:
image_paths,features, cluster_dict=run_dinov2_clustering(motif_folder,60)

加载图像数量：401
开始聚类...

🔹 cluster_6 包含 12 张图像：
  - yuan_Yuan001_2.png
  - qing_Qing005_0.png
  - qing_Qing023_0.png
  - qing_Qing093_8.png
  - qing_Qing095_0.png
  - song_Song021_1.png
  - song_Song028_0.png
  - suitang_宝花纹锦_2_0.png
  - yuan_球路纹朵花绢 _0.png
  - qing_串枝莲花纹缎被面_0.png
  - liaojin_liaojin040_7.png
  - suitang_suitang52_8.png

🔹 cluster_20 包含 30 张图像：
  - yuan_Yuan003_0.png
  - yuan_Yuan024_2.png
  - ming_Ming006_0.png
  - ming_Ming040_5.png
  - ming_Ming041_1.png
  - ming_Ming058_1.png
  - ming_Ming064_3.png
  - ming_Ming076_0.png
  - ming_Ming102_0.png
  - ming_Ming103_0.png
  - ming_Ming104_0.png
  - ming_Ming105_0.png
  - ming_Ming106_0.png
  - ming_Ming107_2.png
  - ming_Ming110_0.png
  - ming_Ming113_0.png
  - qing_Qing061_0.png
  - qing_Qing096_0.png
  - qing_Qing113_0.png
  - qing_Qing126_0.png
  - qing_Qing145_0.png
  - song_Song023_restored_0.png
  - song_Song024_restored_1.png
  - song_Song025_restored_0.png
  - song_Song15_restored_0.png
  - suitang_唐代大窠宝花纹绫_0.png
  - s

In [ ]:
show_images_by_cluster(cluster_dict)